In [ ]:
import nltk
import pandas as pd
import numpy as np
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [ ]:
url = 'https://raw.githubusercontent.com/MuhammedBuyukkinaci/TensorFlow-Sentiment-Analysis-on-Amazon-Reviews-Data/master/dataset/train_amazon.csv'
amazon_Reviews = pd.read_csv(url)
print(amazon_Reviews)
print(amazon_Reviews.shape)

        label                                               text
0           0  Reference Yes, Learning or Instruction No: I p...
1           0  NO!!!!!!I will give it 1 star.: This doll is n...
2           0  Neat Features/Uncomfortable: I bought this wit...
3           1  Progressive-Underground divas??? where's the v...
4           0  The theif who stole a week.: What a stinker! I...
...       ...                                                ...
149995      0  Dubravka Tomsic never recorded the 14 waltzes!...
149996      1  5 year old loves it.: Good purchase for us. He...
149997      0  Busta should've done better...: As a big Papoo...
149998      1  Really Great Soap but...: This soap lathers wo...
149999      0  Just so so: Compared with the sound sensation ...

[150000 rows x 2 columns]
(150000, 2)


In [ ]:
amazon_Reviews['label'].value_counts()

0    75034
1    74966
Name: label, dtype: int64

In [ ]:
amazon_Reviews.dropna(inplace=True)

In [ ]:
amazon_Reviews.describe()

,label
count,150000.000000
mean,0.499773
std,0.500002
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [ ]:
empty=[]
for i, label, review in amazon_Reviews.itertuples():
  if type(review)==str:
    if review.isspace():
      epmty.append(i)

In [ ]:
amazon_Reviews.drop(empty, inplace=True)

In [ ]:
amazon_Reviews['text'].value_counts()

RICHNESS OF AMERICAN HISTORY!: i only wished i could have known about this "GREAT BOOK" back in H.S! The RICHNESS OF THE AMERICAN CULTURE IS ONE THAT EVERY YOUN WOMEN & MEN SHOULD HAVE KNOWLEDGE OF! Things were FOUGHT HARD W/BLOOD & SWEAT TO BE WHERE THIS NATION IS! Every young life will benefit GREATLY FROM THE GREAT FOUNDATION THE CONSTITUTION WAS BUILT UPON! We as American Citizens have lost SO MUCH DO TO THE LACK OF DESIRE TO TEACH OUR CHILDREN THIS WONDERFUL VALUES!!! It will most DEFINETLY HELP SHAPE THE LIVES OF YOUNG PEOPLE! Teach that things are CONQUERED THROUGH HARD WORK! AND GREAT DESIRE TO BECOME THE BEST THAT ONE CAN! NOT A SOCIETY OF YOUNG PEOPLE THAT THINK THEY ARE "ENTITLED" TO EVERY BENEFIT OF OTHER PEOPLE'S HARD LABOR!!!    1
Voice of an Angel!: One of the most beautiful angelic voice in the world! But I don't think this collection shows her best recording pieces.                                                                                                         

In [ ]:
sia.polarity_scores(amazon_Reviews.loc[0]['text'])

{'compound': 0.5106, 'neg': 0.021, 'neu': 0.902, 'pos': 0.077}

In [ ]:
amazon_Reviews.loc[0]['text']

'Reference Yes, Learning or Instruction No: I purchased this book because the customer reviews were fairly positive. But this book is a lot like going on a scavenger hunt. Information is scattered throughout and it is your job to find it. For example, the term "threads" is referred to quite often early on but they don\'t define this term until midway through the book. I found myself jumping all over the place for either a definition or explanation.'

In [ ]:
amazon_Reviews['scores']=amazon_Reviews['text'].apply(lambda review: sia.polarity_scores(review))
amazon_Reviews.head()

,label,text,scores
0,0,"Reference Yes, Learning or Instruction No: I p...","{'neg': 0.021, 'neu': 0.902, 'pos': 0.077, 'co..."
1,0,NO!!!!!!I will give it 1 star.: This doll is n...,"{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'comp..."
2,0,Neat Features/Uncomfortable: I bought this wit...,"{'neg': 0.044, 'neu': 0.775, 'pos': 0.181, 'co..."
3,1,Progressive-Underground divas??? where's the v...,"{'neg': 0.098, 'neu': 0.73, 'pos': 0.172, 'com..."
4,0,The theif who stole a week.: What a stinker! I...,"{'neg': 0.027, 'neu': 0.737, 'pos': 0.236, 'co..."


In [ ]:
amazon_Reviews['compound']=amazon_Reviews['scores'].apply(lambda scores_dict: scores_dict['compound'])
amazon_Reviews.head()

,label,text,scores,compound
0,0,"Reference Yes, Learning or Instruction No: I p...","{'neg': 0.021, 'neu': 0.902, 'pos': 0.077, 'co...",0.5106
1,0,NO!!!!!!I will give it 1 star.: This doll is n...,"{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'comp...",0.3544
2,0,Neat Features/Uncomfortable: I bought this wit...,"{'neg': 0.044, 'neu': 0.775, 'pos': 0.181, 'co...",0.9674
3,1,Progressive-Underground divas??? where's the v...,"{'neg': 0.098, 'neu': 0.73, 'pos': 0.172, 'com...",0.9306
4,0,The theif who stole a week.: What a stinker! I...,"{'neg': 0.027, 'neu': 0.737, 'pos': 0.236, 'co...",0.9896


In [ ]:
amazon_Reviews['compound_score in P/N']=amazon_Reviews['compound'].apply(lambda c: 'pos' if c>=0 else 'neg')
amazon_Reviews.head()

,label,text,scores,compound,compound_score in P/N
0,0,"Reference Yes, Learning or Instruction No: I p...","{'neg': 0.021, 'neu': 0.902, 'pos': 0.077, 'co...",0.5106,pos
1,0,NO!!!!!!I will give it 1 star.: This doll is n...,"{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'comp...",0.3544,pos
2,0,Neat Features/Uncomfortable: I bought this wit...,"{'neg': 0.044, 'neu': 0.775, 'pos': 0.181, 'co...",0.9674,pos
3,1,Progressive-Underground divas??? where's the v...,"{'neg': 0.098, 'neu': 0.73, 'pos': 0.172, 'com...",0.9306,pos
4,0,The theif who stole a week.: What a stinker! I...,"{'neg': 0.027, 'neu': 0.737, 'pos': 0.236, 'co...",0.9896,pos


In [ ]:
amazon_Reviews['compound_score'] = amazon_Reviews['compound_score in P/N'].apply(lambda x: 0 if x=='neg' else 1)
amazon_Reviews.head(20)

,label,text,scores,compound,compound_score in P/N,compound_score
0,0,"Reference Yes, Learning or Instruction No: I p...","{'neg': 0.021, 'neu': 0.902, 'pos': 0.077, 'co...",0.5106,pos,1
1,0,NO!!!!!!I will give it 1 star.: This doll is n...,"{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'comp...",0.3544,pos,1
2,0,Neat Features/Uncomfortable: I bought this wit...,"{'neg': 0.044, 'neu': 0.775, 'pos': 0.181, 'co...",0.9674,pos,1
3,1,Progressive-Underground divas??? where's the v...,"{'neg': 0.098, 'neu': 0.73, 'pos': 0.172, 'com...",0.9306,pos,1
4,0,The theif who stole a week.: What a stinker! I...,"{'neg': 0.027, 'neu': 0.737, 'pos': 0.236, 'co...",0.9896,pos,1
5,0,This movie was terrible!: When I subjected mys...,"{'neg': 0.096, 'neu': 0.744, 'pos': 0.16, 'com...",0.7235,pos,1
6,1,I love wood: I have a few in-laws who benfited...,"{'neg': 0.0, 'neu': 0.675, 'pos': 0.325, 'comp...",0.8910,pos,1
7,1,"Lets me use my own coffee, save K-Cups, and be...","{'neg': 0.043, 'neu': 0.836, 'pos': 0.121, 'co...",0.9180,pos,1
8,0,Good for study purposes: When I first opened t...,"{'neg': 0.09, 'neu': 0.853, 'pos': 0.058, 'com...",-0.7219,neg,0
9,1,Great for Beadweaving!: I copy portions of wor...,"{'neg': 0.057, 'neu': 0.659, 'pos': 0.284, 'co...",0.9531,pos,1


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy_score(amazon_Reviews['label'], amazon_Reviews['compound_score'])

0.71034

In [ ]:
classification_report(amazon_Reviews['label'], amazon_Reviews['compound_score'])

'              precision    recall  f1-score   support\n\n           0       0.87      0.49      0.63     75034\n           1       0.65      0.93      0.76     74966\n\n    accuracy                           0.71    150000\n   macro avg       0.76      0.71      0.70    150000\nweighted avg       0.76      0.71      0.70    150000\n'

In [ ]:
confusion_matrix(amazon_Reviews['label'], amazon_Reviews['compound_score'])

array([[36885, 38149],
       [ 5300, 69666]])

In [ ]:
#Thanks